In [1]:
import torch
import torch.nn.functional as F
import numpy as np

from ssgd import StreamingSGD

# Model definition

A StreamingSGD compatible model now needs to be able to "detach" layers as well as gather input/output and gradients. It also needs a list of layers. See below for implementation example. In the future we want to implement this using PyTorch hooks and modules.

In [2]:
class ExampleNet(torch.nn.Module):
    def __init__(self):
        super(ExampleNet, self).__init__()
        
        self.input_layer = torch.nn.Conv2d(3, 3, kernel_size=3)
        self.layers = [self.input_layer]

        for i in range(5):  # use 9 for 8194 x 8194 images
            self.add_block(i)
        
        final_conv5 = torch.nn.Conv2d(3, 1, kernel_size=10)
        self.add_module("final", final_conv5)
        
        self.layers.extend([final_conv5])
        
    def add_block(self, i):
        conv1 = torch.nn.Conv2d(3, 3, kernel_size=3)
        conv2 = torch.nn.Conv2d(3, 3, kernel_size=3)
        conv3 = torch.nn.Conv2d(3, 3, kernel_size=3)
        maxpool = torch.nn.MaxPool2d(2, stride=2)
        
        self.add_module("conv1-" + str(i), conv1)
        self.add_module("conv2-" + str(i), conv2)
        self.add_module("conv3-" + str(i), conv3)
        self.add_module("maxpool-" + str(i), maxpool)

        self.layers.extend([conv1, conv2, conv3, maxpool])
        
    def backward(self, gradient):
        self.gradients = []
        for i, output in reversed(list(enumerate(self.output))):
            if i < (len(self.output) - 1):
                gradient = self.input[i+1].grad
            output.backward(gradient=gradient, retain_graph=True)
            self.gradients.append(gradient)

    def forward(self, x, stop_index=-1, start_index=0, detach=False):
        if detach:
            self.output = []
            self.input = []
        for i, layer in enumerate(self.layers[start_index:]):
            if detach:
                x = torch.autograd.Variable(x.data, requires_grad=detach)
                self.input.append(x)
            if i == stop_index:
                break
            if i + 1 == len(self.layers[start_index:]):
                x = layer(x)
                x = x.view(-1, 1)
                x = F.sigmoid(x)
            else:
                x = F.relu(layer(x))
            if detach:
                self.output.append(x)
        return x

model = ExampleNet()

Weight initialization; we use positive values to generate large gradients, better for testing if final gradients are correct.

In [3]:
for i, layer in enumerate(model.modules()):
    if isinstance(layer, torch.nn.Conv2d):
        layer.weight.data.fill_(0.04)
        layer.bias.data.zero_()

In [4]:
print(model)

ExampleNet(
  (input_layer): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  (conv1-0): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  (conv2-0): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  (conv3-0): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  (maxpool-0): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
  (conv1-1): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  (conv2-1): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  (conv3-1): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  (maxpool-1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
  (conv1-2): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  (conv2-2): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  (conv3-2): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  (maxpool-2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
  (conv1-3): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  (conv2-3): Conv2d(3, 3, kerne

In [5]:
sCNN = StreamingSGD(model)

# Configurations

In [6]:
stop_index = 13  # use 21 for 8194x8194 images
img_size = 514  # try 8194, see last segment for details

cuda = False  # execute this notebook on the GPU
verbose = True  # enable / disable logging
divide_in = 2  # tip: use 25 for 8194x8194 when memory constraint

# Configure streaming SGD

In [7]:
if cuda:
    model.cuda()

In [8]:
sCNN.configure(model.layers, stop_index, (img_size, img_size, 3), divide_in, cuda, verbose)

Calculating patch boxes...
[1, 3, 512.0, 512.0] [1 1] [ 1.  1.  1.  1.] Conv2d
[1, 3, 510.0, 510.0] [1 1] [ 1.  1.  1.  1.] Conv2d
[1, 3, 508.0, 508.0] [1 1] [ 1.  1.  1.  1.] Conv2d
[1, 3, 506.0, 506.0] [1 1] [ 1.  1.  1.  1.] Conv2d
[1, 3, 253.0, 253.0] [2 2] [ 0.  0.  0.  0.] MaxPool2d
[1, 3, 251.0, 251.0] [1 1] [ 1.  1.  1.  1.] Conv2d
[1, 3, 249.0, 249.0] [1 1] [ 1.  1.  1.  1.] Conv2d
[1, 3, 247.0, 247.0] [1 1] [ 1.  1.  1.  1.] Conv2d
[1, 3, 123.0, 123.0] [2 2] [ 0.  1.  0.  1.] MaxPool2d
[1, 3, 121.0, 121.0] [1 1] [ 1.  1.  1.  1.] Conv2d
[1, 3, 119.0, 119.0] [1 1] [ 1.  1.  1.  1.] Conv2d
[1, 3, 117.0, 117.0] [1 1] [ 1.  1.  1.  1.] Conv2d
[1, 3, 58.0, 58.0] [2 2] [ 0.  1.  0.  1.] MaxPool2d
Embedding divided in patch sizes: (29, 29) 

[1, 3, 328.0, 328.0] [1 1] [ 1.  1.  1.  1.] Conv2d
[1, 3, 326.0, 326.0] [1 1] [ 1.  1.  1.  1.] Conv2d
[1, 3, 324.0, 324.0] [1 1] [ 1.  1.  1.  1.] Conv2d
[1, 3, 322.0, 322.0] [1 1] [ 1.  1.  1.  1.] Conv2d
[1, 3, 161.0, 161.0] [2 2] [ 0.  0.  

# Generate random image and fake label

In the current implementation the whole image needs to be able to fit in memory (RAM).

In [9]:
image = torch.FloatTensor(3, img_size, img_size).normal_(0, 1)
target = torch.FloatTensor(1, 1).fill_(0)

if cuda:
    target = target.cuda()

In [10]:
image_var = torch.autograd.Variable(image).float()

In [11]:
criterion = torch.nn.BCELoss()

In [12]:
output, feature_map = sCNN.forward(image_var)

  0%|          | 0/4 [00:00<?, ?it/s]

Doing forward pass...


100%|██████████| 4/4 [00:00<00:00, 43.75it/s]


In [13]:
output

Variable containing:
 0.9902
[torch.FloatTensor of size 1x1]

In [14]:
loss = criterion(output, torch.autograd.Variable(target)); loss

Variable containing:
 4.6299
[torch.FloatTensor of size 1]

In [15]:
full_gradients = sCNN.backward(image_var, feature_map, loss)

  0%|          | 0/4 [00:00<?, ?it/s]

Doing backward pass...


100%|██████████| 4/4 [00:00<00:00,  8.04it/s]


Filled gradient sizes:
[[  35.   58.   58.]
 [  69.  117.  117.]
 [  69.  119.  119.]
 [  69.  121.  121.]
 [  69.  123.  123.]
 [ 137.  247.  247.]
 [ 137.  249.  249.]
 [ 137.  251.  251.]
 [ 137.  253.  253.]
 [ 274.  506.  506.]
 [ 274.  508.  508.]
 [ 274.  510.  510.]
 [ 274.  512.  512.]] 

Everything filled:
 True


"Everything filled" means that all gradients were reconstructed succesfully!

Save the gradients of the conv2d layer to compare with normal SGD:

In [16]:
streaming_conv_gradients = []

for i, layer in enumerate(model.layers):
    if isinstance(layer, torch.nn.Conv2d):
        if layer.weight.grad is not None:
            streaming_conv_gradients.append(layer.weight.grad.clone()) 

# Compare to normal SGD

Reset the gradients and perform a normal for backward pass.

In [17]:
for i, layer in enumerate(model.layers):
    if isinstance(layer, torch.nn.Conv2d):
        if layer.weight.grad is not None:
            layer.weight.grad.data.zero_()
            layer.bias.grad.data.zero_()

In [18]:
output = model.forward(image_var[None]);

In [19]:
output

Variable containing:
 0.9902
[torch.FloatTensor of size 1x1]

This output should be the same as the streaming SGD output, if so the loss will also be the same:

In [20]:
loss = criterion(output, torch.autograd.Variable(target)); loss

Variable containing:
 4.6299
[torch.FloatTensor of size 1]

Here we do a normal PyTorch backward pass:

In [21]:
loss.backward()

# Compare the gradients of the conv2d layers

In [22]:
normal_conv_gradients = []

for i, layer in enumerate(model.layers):
    if isinstance(layer, torch.nn.Conv2d):
        if layer.weight.grad is not None:
            normal_conv_gradients.append(layer.weight.grad) 

In [23]:
for i in range(len(streaming_conv_gradients)):
    max_diff = np.max(np.abs(streaming_conv_gradients[i].data.cpu().numpy() - 
                      normal_conv_gradients[i].data.cpu().numpy()))
    print("Layer", i, 
          "maximum difference between actual gradient:", max_diff)

Layer 0 maximum difference between actual gradient: 2.74181e-06
Layer 1 maximum difference between actual gradient: 2.02656e-06
Layer 2 maximum difference between actual gradient: 1.43051e-06
Layer 3 maximum difference between actual gradient: 2.5034e-06
Layer 4 maximum difference between actual gradient: 9.53674e-07
Layer 5 maximum difference between actual gradient: 1.19209e-06
Layer 6 maximum difference between actual gradient: 1.19209e-06
Layer 7 maximum difference between actual gradient: 5.96046e-07
Layer 8 maximum difference between actual gradient: 3.57628e-07
Layer 9 maximum difference between actual gradient: 3.57628e-07
Layer 10 maximum difference between actual gradient: 0.0
Layer 11 maximum difference between actual gradient: 0.0
Layer 12 maximum difference between actual gradient: 0.0
Layer 13 maximum difference between actual gradient: 0.0
Layer 14 maximum difference between actual gradient: 0.0
Layer 15 maximum difference between actual gradient: 0.0
Layer 16 maximum di

As you can see the difference of the gradients of the conv2d layers between the methods is (almost) numerically equivalent. The small differences are because of loss of significance with the floating points calculations. 

---

# Things to try:

* Use doubles instead of floats to reduce the difference (use model.double() and image_var.double())
* Make the image bigger than would fit on a GPU 
    - e.g. 8194x8194, make sure to add 3 more blocks in the model (see comments)
* If you want you can compare the reconstructed input gradients of each layer: 
    - pass fill_gradient=True in backward() function
    - compare full_gradients with self.model.gradients after the full model backward pass.
* For testing purposes the number of filters is small in this notebook, try increasing them